In [2]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
from multiprocessing import Pool, Process
import os


In [15]:
def get_all_page(urls, pattern, url_base):

    if pattern in urls:
        return(urls)
    
    html = requests.get(pattern)
    soup = BeautifulSoup(html.content, "html.parser")
    last_page = soup.find("a", {"title" : "Dernière page"})["page"]

    iter = soup.find_all("div", {"class" : re.compile("m3-pagination print_hidden|print_hidden pagination")})
    for item in iter:
        lst = re.findall("url.*?;", item.find_next_sibling().text)
        nw_url = ""
        if len(lst) > 0:
            for item in lst:
                if re.search("querystring", item):
                    continue
                else:
                    item = re.sub("url =|url \+=|url", "", item)
                    item = item.replace(";", "").replace("\"", "").strip()
                    item = item.split(" + ")
                for part in item:
                    nw_url += part
            match = re.compile("\$\(this\)\.attr\(page\)")
            for i in range(1, int(last_page)+1):
                url = urljoin(url_base, re.sub(match,str(i), nw_url))
                if url not in urls:
                    urls.append(url)
                    
        else:
            for i in range(1, int(last_page)+1):
                urls.append(urljoin(url_base, re.sub("page=.", f"page={i}", pattern)))
    
    return(urls)

In [18]:
def getAllUrl(url, url_base, urls, errors):
    
    page = requests.get(url)

    if page.status_code == 200:
        soup = BeautifulSoup(page.content, "html.parser")
        for anchor in soup.find_all("a"):
            if "href" in anchor.attrs:
                if anchor["href"] == "JavaScript:void(0);" and anchor["class"] == "lien-page":
                    try:
                        urls.extend(get_all_page(urls), url, url_base)
                    except:
                        continue
                absolute_url = urljoin(url_base, anchor["href"])
                if "download" not in absolute_url and absolute_url not in urls:
                    if re.search(url_base, absolute_url):
                        urls.append(absolute_url)

        # Export des pages html ?

    else:
        print(" ", page.status_code)
        errors.append(url)

    return(urls)

In [5]:
def read_urls(EXPORT_FILE):
    with open(EXPORT_FILE, "r") as f:
        urls = f.read().split("\n") 
    print(len(urls))
    return(urls)

In [22]:

EXPORT_FILE = "M3urls3.txt"

if __name__ == "__main__":
    
    url_base = 'https://www.millenaire3.com'
    errors = []
    urls = get_all_page(urls = [], 
                        pattern="https://www.millenaire3.com/m3_search?page=1&sort_by=0&SearchText=&modeRecherche=1&titre=&auteurId=&auteurText=&100%25=&100%25_new_value=true&date_debut=01%2F01%2F1900&date_debut_mobile=&date_fin=31%2F12%2F2025&date_fin_mobile=&identifiers=&rubriques=",
                        url_base = url_base)
    # if EXPORT_FILE in os.listdir():
    #     urls = read_urls(EXPORT_FILE)
    # else:
    #     urls = [url_base]

    i = 0# On peut changer la valeur de départ
    while i <= len(urls)-1:
        if re.search("/partage_email/|/generated_pdf/|/toutes-les-videos", urls[i]):
            errors.append(urls[i])
            print(f"\r{i} / {len(urls)}")
        else:
            getAllUrl(urls[i], url_base, urls, errors)
            i += 1
            print(f"\r{i} / {len(urls)} : {urls[i-1]}", end = "")

with open(EXPORT_FILE, "w", encoding="utf-8") as f:
    for url in urls:
        f.write(url+"\n")

2507 / 10276 : https://www.millenaire3.com/ressources/conception-eco-culturelle-d-un-centre-de-rechercheuesetudeepreuve-des-exigences-renouvelees-des-usagers-en-francenpied-et-les-deux-roues-soient-plus-efficaces1%2F12%2F2025&date_fin_mobile=&identifiers=&rubriques=  404
3854 / 13162 : https://www.millenaire3.com/ressources/Les-discriminations-raciales-Comment-agirvernanceent-de-la-population-et-ses-consequencesn-debatnnaise-specificites-et-evolution-par-la-cofremcaeslyon-et-turin

In [34]:
urls = read_urls(EXPORT_FILE)

12367


In [20]:
tmp = get_all_page([], 
             "https://www.millenaire3.com/toutes-les-videos",
             "https://www.millenaire3.com/")

errors = []

getAllUrl("https://www.millenaire3.com/toutes-les-videos", "https://www.millenaire3.com/", tmp, errors)

['https://www.millenaire3.com/toutes-les-videos?p=1',
 'https://www.millenaire3.com/toutes-les-videos?p=2',
 'https://www.millenaire3.com/toutes-les-videos?p=3',
 'https://www.millenaire3.com/toutes-les-videos?p=4',
 'https://www.millenaire3.com/toutes-les-videos?p=5',
 'https://www.millenaire3.com/toutes-les-videos?p=6',
 'https://www.millenaire3.com/toutes-les-videos?p=7',
 'https://www.millenaire3.com/toutes-les-videos?p=8',
 'https://www.millenaire3.com/toutes-les-videos?p=9',
 'https://www.millenaire3.com/toutes-les-videos?p=10',
 'https://www.millenaire3.com/toutes-les-videos?p=11',
 'https://www.millenaire3.com/toutes-les-videos?p=12',
 'https://www.millenaire3.com/',
 'https://www.millenaire3.com/#content',
 'https://www.millenaire3.com/#main-menu',
 'https://www.millenaire3.com/#block-search',
 'https://www.millenaire3.com/nos-flux-rss',
 'https://www.millenaire3.com/societe',
 'https://www.millenaire3.com/modes-d-action',
 'https://www.millenaire3.com/transformation-urbaine',